# Using Virtuoso Medium v2 on SageMaker through the Hugging Face hub

## 1. Import dependencies

In [ ]:
%%sh
pip install -qU sagemaker huggingface_hub

In [ ]:
import datetime
import json
import pprint

import boto3
import sagemaker
from IPython.display import Markdown, display
from sagemaker import get_execution_role
from sagemaker.djl_inference.model import DJLModel
from sagemaker_streaming import print_event_stream

In [ ]:
role = get_execution_role()
sagemaker_session = sagemaker.Session()
runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

In this example, we're deploying Virtuoso Medium v2 on a SageMaker real-time endpoint hosted on a GPU instance. If you need general information on real-time inference with Amazon SageMaker, please refer to the SageMaker [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

The endpoint runs a DJLServing [Deep Learning Container](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-models-frameworks-djl-serving.html), powered by vLLM. vLLM enables high-performance text generation for the most popular open-source language models. 

The endpoint configuration focuses on cost efficiency. It uses a [g6e.12xlarge](https://aws.amazon.com/ec2/instance-types/g6e/) instance. This instance has four NVDIA L40S GPU, with 192GB of GPU RAM. Virtuoso Medium v2 has 32.8 billion 16-bit parameters, which can easily fit without the need for quantization.

#### OpenAI compatibility

The [OpenAI Messages API](https://huggingface.co/docs/text-generation-inference/messages_api) is available. This alllows you to invoke the endpoint in the same way you would invoke an OpenAI model. Likewise, the output format is identical to the OpenAI models.

### A. Define the endpoint configuration

In [ ]:
model_id = "arcee-ai/Virtuoso-Medium-v2"
endpoint_name_prefix = "Virtuoso-Medium-v2"

In [ ]:
# g6e endpoint

real_time_inference_instance_type = "ml.g6e.12xlarge"

### B. Create the endpoint

In [ ]:
model = DJLModel(model_id=model_id, role=role, env=None)

# create a unique endpoint name
timestamp = "{:%Y-%m-%d-%H-%M-%S}".format(datetime.datetime.now())
endpoint_name = f"{endpoint_name_prefix}-{timestamp}"
print(f"Deploying endpoint {endpoint_name}")

# deploy the model
response = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type,
    endpoint_name=endpoint_name,
    model_data_download_timeout=900,
    container_startup_health_check_timeout=900,
)

Once the endpoint is in service, you will be able to perform real-time inference.

### C. Define a test payload

In [ ]:
model_sample_input = {
    "messages": [
        {"role": "system", "content": "You are a friendly and helpful AI assistant."},
        {
            "role": "user",
            "content": "Suggest 5 names for a new neighborhood pet food store. Names should be short, fun, easy to remember, and respectful of pets. \
        Explain why customers would like them.",
        },
    ],
    "max_tokens": 1024,
}

### D. Perform real-time inference

In [ ]:
%%time
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(model_sample_input),
)

output = json.loads(response["Body"].read().decode("utf8"))

### E. Visualize output

We can print the raw JSON output in OpenAI format.

In [ ]:
pprint.pprint(output)

We can also print the generated output with Markdown formatting.

In [ ]:
display(Markdown(output["choices"][0]["message"]["content"]))

Here are some more examples. Please feel free to tweak them and add your own!

### F. Perform streaming inference

In [ ]:
prompt = """Please write a friendly marketing pitch for a new SaaS AI platform called Arcee Cloud.
We will send this pitch by email to business and technical decision-makers, so make it sound exciting yet professional.
The contact email is sales@arcee.ai. Feel free to use emojis as appropriate.
Arcee Cloud makes it simple for enterprise users to tailor open-source small language models to their own domain knowledge,
in order to build high-quality, cost-effective and secure AI solutions."""

model_sample_input = {
    "messages": [
        {
            "role": "system",
            "content": "You are a friendly and helpful Marketing Manager working at Arcee.ai.",
        },
        {"role": "user", "content": prompt},
    ],
    "max_tokens": 1024,
    "stream": True
}

response = runtime_sm_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(model_sample_input),
    ContentType='application/json'
)

print_event_stream(response['Body'])

In [ ]:
model_sample_input = {
    "messages": [
        {
            "role": "system",
            "content": "As a friendly technical assistant engineer, answer the question in detail.",
        },
        {"role": "user", "content": "Why are transformers better models than LSTM? Explain step by step."},
    ],
    "max_tokens": 1024,
    "stream": True
}

response = runtime_sm_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(model_sample_input),
    ContentType='application/json'
)

print_event_stream(response['Body'])

In [ ]:
model_sample_input = {
    "messages": [
        {
            "role": "system",
            "content": "You are Darlene, a friendly and helpful salesperson \
        working at Crystal River Classic Bikes, a classic motorcycle dealership in central Florida.",
        },
        {
            "role": "user",
            "content": "Using English, write a personalized customer email to get \
        them to sign up for a test ride on the new 2025 motorcycles that are visible at the dealership. \
        Tone should be warm and personal, make sure to weave in the customer information below. \
        Wyatt, your chief mechanic and road captain, has just won the 2024 State Award for Best Mechanic. \
        \
        Customer information:\
        - name: Julien \
        - last visit: 6 months ago for bike service \
        - Owns 2 bikes, a 2002 sporty bike and a 2007 cruiser \
        ",
        },
    ],
    "max_tokens": 1024,
    "stream": True
}

response = runtime_sm_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(model_sample_input),
    ContentType='application/json'
)

print_event_stream(response['Body'])

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

## 4. Clean-up

Please don't forget to run the cells below to delete all resources and avoid unecessary charges.

### A. Delete the endpoint

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()

Thank you for trying out Virtuoso Medium v2 on SageMaker. We have only scratched the surface of what you can do with this model.

We'd be happy to hear from you, learn more about your use case, and help you build your next AI-driven solution. Please reach out to julien@arcee.ai.